# Orders

✏️ Now, we will investigate the **orders**, and their associated **review score**. 

⚠️ We also want to filter out "non-delivered" orders, unless explicitly specified, otherwise we cannot compute the potential delays.

In [1]:
# Auto reload imported module 
%load_ext autoreload
%autoreload 2

In [2]:
# Import usual modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Import olist data
from olist.data import Olist
olist = Olist()
data = olist.get_data()

In [4]:
# list of accessible dataset
data.keys()

dict_keys(['sellers', 'order_reviews', 'order_items', 'customers', 'orders', 'order_payments', 'product_category_name_translation', 'products', 'geolocation'])

In [5]:
orders = data['orders'].copy()
assert(orders.shape == (99441, 8))

## 1. Data Wrangling for order.py

As per previous exploration, we found several orders have delivery issues. We'll further dive into this aspect by first getting the information of the **wait_time**, **expected_wait_time**, **delayed_vs_expected** and **order_status** of each individual orders to identify if there's any trend.

In [6]:
orders.order_status.unique()

array(['delivered', 'invoiced', 'shipped', 'processing', 'unavailable',
       'canceled', 'created', 'approved'], dtype=object)

In [60]:
# We'll load the orders datafile and convert them to the correct format before further processing
orders = orders[orders['order_status']=='delivered'].copy()
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
orders['order_estimated_delivery_date'] = pd.to_datetime(orders['order_estimated_delivery_date'])

In [73]:
# wait_time, expected_wait_time, delay_vs_expected are calculated based on existing columns for further analysis
orders['wait_time'] = (orders['order_delivered_customer_date'] - orders['order_purchase_timestamp']).dt.days
orders['expected_wait_time'] = (orders['order_estimated_delivery_date'] - orders['order_purchase_timestamp']).dt.days
orders['delay_vs_expected'] = orders['wait_time'] - orders['expected_wait_time']
orders.loc[orders['delay_vs_expected']<0,'delay_vs_expected'] = 0

In [72]:
# Final dataframe will be only the selected columns as shown below
orders['delay_vs_expected'].fillna(value=0,inplace=True)
orders['delay_vs_expected'] = orders['delay_vs_expected'].astype(int)
orders.filter(items=['order_id','wait_time','expected_wait_time','delay_vs_expected','order_status'])

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status
0,e481f51cbdc54678b7cc49136f2d6af7,8.0,15,0,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,13.0,19,0,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,9.0,26,0,delivered
3,949d5b44dbf5de918fe9c16f97b45f8a,13.0,26,0,delivered
4,ad21c59c0840e6cb83a9ceb5573f8159,2.0,12,0,delivered
...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,8.0,18,0,delivered
99437,63943bddc261676b46f01ca7ac2f7bd8,22.0,23,0,delivered
99438,83c1379a015df1e13d02aae0204711ab,24.0,30,0,delivered
99439,11c177c8e97725db2631073c19f07b62,17.0,37,0,delivered


In [16]:
orders.isna().sum()

order_id                          0
customer_id                       0
order_status                      0
order_purchase_timestamp          0
order_approved_at                14
order_delivered_carrier_date      2
order_delivered_customer_date     8
order_estimated_delivery_date     0
wait_time                         8
expected_wait_time                0
delay_vs_expected                 8
dtype: int64

👀 The code above is checked, confirmed and commit it to `olist/order.py`  

🧪 Now, the python file is rerun and tested as per the following cell 👇 

In [20]:
# Test your code here
from olist.order import Order
b = Order().get_wait_time()
b

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status
0,e481f51cbdc54678b7cc49136f2d6af7,8.0,15.0,0.0,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,13.0,19.0,0.0,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,9.0,26.0,0.0,delivered
3,949d5b44dbf5de918fe9c16f97b45f8a,13.0,26.0,0.0,delivered
4,ad21c59c0840e6cb83a9ceb5573f8159,2.0,12.0,0.0,delivered
...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,8.0,18.0,0.0,delivered
99437,63943bddc261676b46f01ca7ac2f7bd8,22.0,23.0,0.0,delivered
99438,83c1379a015df1e13d02aae0204711ab,24.0,30.0,0.0,delivered
99439,11c177c8e97725db2631073c19f07b62,17.0,37.0,0.0,delivered


In [21]:
b.isna().sum()

order_id              0
wait_time             8
expected_wait_time    0
delay_vs_expected     8
order_status          0
dtype: int64

### Data Wrangling for function `get_review_score`


In [74]:
reviews = data['order_reviews'].copy()
assert(reviews.shape == (99224,7))

In [75]:
reviews.dtypes

review_id                  object
order_id                   object
review_score                int64
review_comment_title       object
review_comment_message     object
review_creation_date       object
review_answer_timestamp    object
dtype: object

In [76]:
reviews.isna().sum()

review_id                      0
order_id                       0
review_score                   0
review_comment_title       87656
review_comment_message     58247
review_creation_date           0
review_answer_timestamp        0
dtype: int64

In [25]:
reviews['dim_is_five_star'] = np.where(reviews['review_score']==5,1,0)
reviews['dim_is_five_star'].head(3)

0    0
1    1
2    1
Name: dim_is_five_star, dtype: int64

In [26]:
reviews['dim_is_one_star'] = np.where(reviews['review_score']==1,1,0)
reviews['dim_is_one_star'].head(3)

0    0
1    0
2    0
Name: dim_is_one_star, dtype: int64

In [27]:
reviews.filter(items=['order_id', 'dim_is_five_star', 'dim_is_one_star', 'review_score'])

,order_id,dim_is_five_star,dim_is_one_star,review_score
0,73fc7af87114b39712e6da79b0a377eb,0,0,4
1,a548910a1c6147796b98fdf73dbeba33,1,0,5
2,f9e4b658b201a9f2ecdecbb34bed034b,1,0,5
3,658677c97b385a9be170737859d3511b,1,0,5
4,8e6bfb81e283fa7e4f11123a3fb894f1,1,0,5
...,...,...,...,...
99219,2a8c23fee101d4d5662fa670396eb8da,1,0,5
99220,22ec9f0669f784db00fa86d035cf8602,1,0,5
99221,55d4004744368f5571d1f590031933e4,1,0,5
99222,7725825d039fc1f0ceb7635e3f7d9206,0,0,4


After we confirmed everything is correct, the code is then commit to `olist/order.py`

In [28]:
# Testing
from olist.order import Order
Order().get_review_score()

,order_id,dim_is_five_star,dim_is_one_star,review_score
0,73fc7af87114b39712e6da79b0a377eb,0,0,4
1,a548910a1c6147796b98fdf73dbeba33,1,0,5
2,f9e4b658b201a9f2ecdecbb34bed034b,1,0,5
3,658677c97b385a9be170737859d3511b,1,0,5
4,8e6bfb81e283fa7e4f11123a3fb894f1,1,0,5
...,...,...,...,...
99219,2a8c23fee101d4d5662fa670396eb8da,1,0,5
99220,22ec9f0669f784db00fa86d035cf8602,1,0,5
99221,55d4004744368f5571d1f590031933e4,1,0,5
99222,7725825d039fc1f0ceb7635e3f7d9206,0,0,4


### c) `get_number_products`:

In [29]:
data.keys()

dict_keys(['sellers', 'order_reviews', 'order_items', 'customers', 'orders', 'order_payments', 'product_category_name_translation', 'products', 'geolocation'])

In [30]:
data['orders'].head(3)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00


In [31]:
data['order_items'].head(3)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.0,17.87


In [32]:
data['products'].head(3)

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0


In [33]:
product_order = data['order_items'].copy()
product_order

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


In [34]:
a = product_order.groupby(by = 'order_id').count()

In [35]:
a.filter(items=['order_id','price'])
a.reset_index(inplace=True)
a.rename(columns= {'price':'test'})

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,test,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,1,1,1,1,1
1,00018f77f2f0320c557190d7a144bdd3,1,1,1,1,1,1
2,000229ec398224ef6ca0657da4fc703e,1,1,1,1,1,1
3,00024acbcdf0a6daa1e931b038114c75,1,1,1,1,1,1
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,1,1,1,1,1
...,...,...,...,...,...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,1,1,1,1,1,1
98662,fffcd46ef2263f404302a634eb57f7eb,1,1,1,1,1,1
98663,fffce4705a9662cd70adb13d4a31832d,1,1,1,1,1,1
98664,fffe18544ffabc95dfada21779c9644f,1,1,1,1,1,1


In [36]:
a.seller_id.unique()

array([ 1,  2,  3,  4,  5,  6,  8,  7, 20, 12, 10, 13, 15, 11, 14, 21,  9])

The dataframe is then commit to `olist/order.py` and loaded from the .py file for reconfirmation.

### d) `get_number_sellers`:


In [37]:
num_sellers = data['order_items'].copy()
num_sellers.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [38]:
num_sellers.shape

(112650, 7)

In [39]:
a = num_sellers.groupby(by=['order_id','seller_id']).count()
a.shape

(100010, 5)

In [40]:
a.reset_index(inplace=True)
a = a.groupby(by=['order_id']).count()
a.reset_index(inplace=True)
a = a.filter(items=['order_id','seller_id'])
a.rename(columns = {'seller_id':'number_of_sellers'},inplace=True)
a

,order_id,number_of_sellers
0,00010242fe8c5a6d1ba2dd792cb16214,1
1,00018f77f2f0320c557190d7a144bdd3,1
2,000229ec398224ef6ca0657da4fc703e,1
3,00024acbcdf0a6daa1e931b038114c75,1
4,00042b26cf59d7ce69dfabb4e55b4fd9,1
...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,1
98662,fffcd46ef2263f404302a634eb57f7eb,1
98663,fffce4705a9662cd70adb13d4a31832d,1
98664,fffe18544ffabc95dfada21779c9644f,1


In [41]:
a.shape

(98666, 2)

### e) `get_price_and_freight`

In [42]:
p_f = data['order_items'].copy()
p_f.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [43]:
p_f = p_f.groupby(by='order_id').sum()
p_f.reset_index(inplace=True)
p_f = p_f.filter(items=['order_id','price','freight_value'])

In [44]:
data['order_items'].shape

(112650, 7)

### e) `get_distance_seller_customer` 

In [45]:
orders = data['orders'].copy()
orders.head(3)
ordersF = orders.filter(items=['order_id','customer_id'])

In [46]:
customers = data['customers'].copy()
customers.head(3)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP


In [47]:
seller = data['sellers'].copy()
seller.head(3)

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ


In [48]:
items = data['order_items'].copy()
items.head(3)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.0,17.87


In [49]:
location = data['geolocation'].copy()
location.head(3)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP


## 2. All at once: `get_training_data`

❓ Finally, there'll be a `get_training_data` that gathers all previous order features in one table

In [50]:
wait = Order().get_wait_time()
wait.head(3)

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status
0,e481f51cbdc54678b7cc49136f2d6af7,8.0,15.0,0.0,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,13.0,19.0,0.0,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,9.0,26.0,0.0,delivered


In [51]:
wait.isna().sum()

order_id              0
wait_time             8
expected_wait_time    0
delay_vs_expected     8
order_status          0
dtype: int64

In [52]:
review = Order().get_review_score()
review.head(3)

,order_id,dim_is_five_star,dim_is_one_star,review_score
0,73fc7af87114b39712e6da79b0a377eb,0,0,4
1,a548910a1c6147796b98fdf73dbeba33,1,0,5
2,f9e4b658b201a9f2ecdecbb34bed034b,1,0,5


In [53]:
product = Order().get_number_products()
product.head(3)

,order_id,number_of_products
0,00010242fe8c5a6d1ba2dd792cb16214,1
1,00018f77f2f0320c557190d7a144bdd3,1
2,000229ec398224ef6ca0657da4fc703e,1


In [54]:
seller = Order().get_number_sellers()
seller.head(3)

,order_id,number_of_sellers
0,00010242fe8c5a6d1ba2dd792cb16214,1
1,00018f77f2f0320c557190d7a144bdd3,1
2,000229ec398224ef6ca0657da4fc703e,1


In [55]:
pnf = Order().get_price_and_freight()
pnf.head(3)

,order_id,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,58.9,13.29
1,00018f77f2f0320c557190d7a144bdd3,239.9,19.93
2,000229ec398224ef6ca0657da4fc703e,199.0,17.87


In [56]:
result = wait.merge(review, on = 'order_id', how='inner').merge(product, on = 'order_id', how='inner')
result = result.merge(seller, on = 'order_id', how='inner').merge(pnf, on = 'order_id', how='inner')
result.head()

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status,dim_is_five_star,dim_is_one_star,review_score,number_of_products,number_of_sellers,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,8.0,15.0,0.0,delivered,0,0,4,1,1,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,13.0,19.0,0.0,delivered,0,0,4,1,1,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,9.0,26.0,0.0,delivered,1,0,5,1,1,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,13.0,26.0,0.0,delivered,1,0,5,1,1,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,2.0,12.0,0.0,delivered,1,0,5,1,1,19.90,8.72


In [57]:
result.dropna()

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status,dim_is_five_star,dim_is_one_star,review_score,number_of_products,number_of_sellers,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,8.0,15.0,0.0,delivered,0,0,4,1,1,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,13.0,19.0,0.0,delivered,0,0,4,1,1,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,9.0,26.0,0.0,delivered,1,0,5,1,1,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,13.0,26.0,0.0,delivered,1,0,5,1,1,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,2.0,12.0,0.0,delivered,1,0,5,1,1,19.90,8.72
...,...,...,...,...,...,...,...,...,...,...,...,...
96356,9c5dedf39a927c1b2549525ed64a053c,8.0,18.0,0.0,delivered,1,0,5,1,1,72.00,13.08
96357,63943bddc261676b46f01ca7ac2f7bd8,22.0,23.0,0.0,delivered,0,0,4,1,1,174.90,20.10
96358,83c1379a015df1e13d02aae0204711ab,24.0,30.0,0.0,delivered,1,0,5,1,1,205.99,65.02
96359,11c177c8e97725db2631073c19f07b62,17.0,37.0,0.0,delivered,0,0,2,2,1,359.98,81.18


In [58]:
result.isnull().sum()

order_id              0
wait_time             8
expected_wait_time    0
delay_vs_expected     8
order_status          0
dim_is_five_star      0
dim_is_one_star       0
review_score          0
number_of_products    0
number_of_sellers     0
price                 0
freight_value         0
dtype: int64